In [ ]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
# Cargar datos desde archivos JSON

with open('user_item_data.json', 'r') as f:
    user_item_data = json.load(f)

with open('review_data.json', 'r') as f:
    review_data = json.load(f)

with open('bundle_data.json', 'r') as f:
    bundle_data = json.load(f)

In [ ]:
# Crear DataFrames
user_item_df = pd.DataFrame(user_item_data['items'])
review_df = pd.DataFrame([review_data])
bundle_df = pd.DataFrame(bundle_data['items'])

# Preprocesamiento

# Convertir los datos de revisión en un DataFrame adecuado para Surprise
review_df = review_df.rename(columns={'username': 'userID', 'product_id': 'itemID', 'text': 'rating'})
review_df['rating'] = 1  # Añadir una columna de calificaciones ficticias para que el formato sea adecuado
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(review_df[['userID', 'itemID', 'rating']], reader)

# Entrenar el modelo con todos los datos disponibles
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Función de recomendación
def recommend(userID, user_item_df, review_df, bundle_df, algo, n=10):
    # Obtener los items que el usuario ya ha visto
    user_items = user_item_df[user_item_df['item_id'].isin(review_df['itemID'])]['item_id'].tolist()
    user_item_names = user_item_df[user_item_df['item_id'].isin(review_df['itemID'])]['item_name'].tolist()


    # Generar predicciones para los items no vistos
    all_items = bundle_df['item_id'].unique()
    recommendations = []

    for itemID in all_items:
        if itemID not in user_items:
            pred = algo.predict(userID, itemID)
            recommendations.append((itemID, pred.est))

    # Ordenar las recomendaciones por calificación predicha
    recommendations.sort(key=lambda x: x[1], reverse=True)

    # Seleccionar los top n items
    top_recommendations = recommendations[:n]
    recommended_items = [item[0] for item in top_recommendations]

    # Obtener nombres de los items recomendados
    recommended_item_names = bundle_df[bundle_df['item_id'].isin(recommended_items)]['item_name'].tolist()

    return recommended_item_names

In [ ]:
# Ejemplo de uso
userID = '76561198091973414'  # ID de usuario del ejemplo
recommendations = recommend(userID, user_item_df, review_df, bundle_df, algo, n=3)
print(f"Recomendaciones para el usuario {userID}: {recommendations}")